# 單層感知器TensorBoard

In [1]:
import tensorflow as tf
import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")


with tf.variable_scope("wb"):
    W = tf.get_variable("w", [784, 10],
        initializer=tf.random_normal_initializer())
    b = tf.get_variable("b", [10],
        initializer=tf.random_normal_initializer())
    
y = tf.nn.softmax(tf.matmul(x_, W) + b)



with tf.variable_scope("cross_entropy") as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

with tf.variable_scope("trainer") as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    trainer = optimizer.minimize(cross_entropy)

with tf.variable_scope("evaluator") as scope:
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


summ_W = tf.histogram_summary("weights", W)
summ_b = tf.histogram_summary("biases", b)
summ_ce = tf.scalar_summary("cross_entropy", cross_entropy)
summ_acc = tf.scalar_summary("accuracy", accuracy)

summ_merged = tf.merge_summary([summ_W, summ_b, summ_ce])


with tf.name_scope("initializer") as scope:
    init = tf.initialize_all_variables()


In [3]:
sess = tf.Session()
sess.run(init)
writer = tf.train.SummaryWriter("./", sess.graph_def)


for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(trainer,  feed_dict={x_: batch_xs, y_: batch_ys})
    summ_str = sess.run(summ_merged,feed_dict={x_:batch_xs,y_:batch_ys})
    writer.add_summary(summ_str, i)
    if (i+1)%5 == 0:
        summary_str = sess.run(summ_acc,feed_dict={x_:mnist.test.images,y_:mnist.test.labels})
        writer.add_summary(summ_str, i)

